In [3]:
#Importing dataset from sklearn

from sklearn import datasets
from sklearn import metrics

iris = datasets.load_breast_cancer() #dataset loading
X = iris.data               #Features stored in X 
y = iris.target             #Class variable

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Installing xgboost
!pip install xgboost

from xgboost import XGBClassifier

clf = XGBClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(metrics.classification_report(y_test, y_pred))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
              precision    recall  f1-score   support

           0       0.98      0.93      0.95        43
           1       0.96      0.99      0.97        71

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114



In [4]:
# 이름과 학번을 작성해주세요
__author__ = "박경원"
__id__ = "2017311362"

import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor


class my_XGB():
    def __init__(self):
        """
        이 함수는 수정할 필요가 없습니다.
        """

        self.author = __author__
        self.id = __id__

        self.boosting_N = 5
        self.weak_classifier = []
        self.score = None
        self.learning_rate = 0.4
        

    def sigmoid(self, x):
        """
        주어진 x에 대하여 sigmoid 활성화 함수를 코딩하세요.
        단, 파이썬 패키지인 math를 불러와서 사용할 수 없습니다. numpy만을 사용하여 코딩하세요.
        """
        #문제 1: x에 대한 sigmoid 결과 값을 return 하세요. 
        return 1 / (1 + np.exp(-x))        # 시그모이드 함수 공식 : 1/(1+exp(-x)) = exp(x)/(exp(x)+1)

    def first_order(self, y, x):
        """
        이 함수는 수정할 필요가 없습니다.
        
        본 함수는 Gradient를 연산하는 함수이며, loss를 근사하기 위해 사용됩니다.
        """
        return (y - self.sigmoid(x))

    def second_order(self, x):
        """
        이 함수는 수정할 필요가 없습니다.
        
        본 함수는 Hessian을 연산하는 함수이며, 앞선 gradient를 미분한 값입니다.
        2차 근사함으로써 더욱 loss를 최소화하기 쉽게 합니다.
        """
        return (self.sigmoid(x) * (1 - self.sigmoid(x)))
        

    def fit(self, X_train, y_train):
        """
        본 함수는 X_train, y_train를 활용하여 훈련하는 과정을 코딩하는 부분입니다.

            설정한 boosting 횟수인 self.boosting_N만큼 훈련을 반복합니다.
            매 반복마다 임의의 decision tree 생성 한 후, 설정한 가중치를 기반으로 모델을 학습하고,
            학습된 Classifier는 self.weak_classifier 리스트에 저장되며,
            해당 Classifier의 예측 값은 self.learning rate를 반영하여 기존의 self.score와 합산됩니다.
        
        """
        self.score = np.zeros(X_train.shape[0]) # 점수 초기화
        sample_weights = [1/len(y_train) for i in range(len(y_train))] # 임의의 가중치 할당
        y_new = y_train
        
        for i in range(self.boosting_N):
            # 임의의 decision tree 생성 한 후, 설정한 가중치를 기반으로 모델 학습
            # 유연한 가중치 부여 및 분류를 위해 regressor 호출
            # 학습된 Classifier는 self.weak_classifier 리스트에 저장
            new_dt = DecisionTreeRegressor()
            new_dt.fit(X_train, y_new, sample_weight=sample_weights) 
            self.weak_classifier.append(new_dt) 
            
            #문제 2: 방금 학습된 DT모델의 X_train 데이터에 대한 prediction score를 도출하세요.
            new_score = new_dt.predict(X_train)     
            # 앞서 만든 DecisionTreeRegressor() 를 이용하여 도출

            #문제 3: 방금 구한 new_score에 learning rate를 반영(곱 연산)하여 self.score를 업데이트(합 연산)하세요.
            self.score = self.score + (1+self.learning_rate) * new_score      
            # 곱연산은 각 수에 1을 더해서 곱하고, 합연산에 의해 각 수를 다 더해준다.     

            #문제 4: 앞선 모델의 gradient를 도출하기 위해 None 부분에 알맞은 변수를 입력하세요.
            grad = self.first_order(y_train, self.score)     

            #문제 5: 앞선 모델의 hessian을 도출하기 위해 None 부분에 알맞은 변수를 입력하세요.
            hess = self.second_order(self.score)       

            # gradient와 hessian을 반영하여 y_new와 가중치 업데이트
            y_new = grad/hess
            sample_weights = hess

    def predict(self, X_test):
        """
        본 함수는 X_test가 주어졌을 때, fit을 통해 계산된 classifier 리스트를 불러와 각 모델의 예측값을 합산하여 최종 도출하는 함수입니다.
        """
        pred = np.zeros(X_test.shape[0])
        #문제 6: fit에서 저장된 weak_classifier들을 불러와 learning rate를 반영(곱 연산)한 prediction 값(합 연산)을 업데이트하세요.
        for w_classifier in self.weak_classifier:
            pred = pred + (1+self.learning_rate)*w_classifier.predict(X_test)
        # learning_rate를 곱 연산한 후 predict 값을 pred에 더하여 값을 구한다.
        
        probas = self.sigmoid(np.full((X_test.shape[0], 1), 1).flatten().astype('float64') + pred)
        #문제 7: probability가 전체 probability 평균보다 초과되면 1, 아닌경우 0으로 prediction 라벨을 할당하세요.
        pred_label = np.where(probas >  np.mean(probas),1,0) #np조건문을 사용하여 평균보다 크면 1, 작으면 0으로 할당 
        return pred_label


    def get_accuracy(self, y_true, y_pred):
        """
        본 함수는 수정할 필요 없는 함수입니다.
        정확도를 연산하기 위한 함수입니다.
        """
        return np.count_nonzero(y_true == y_pred) / len(y_true)

    def score(self, x, y):
        """
        본 함수는 수정할 필요 없는 함수입니다.
        5fold cross validation 함수를 사용하기 위해 필요한 함수입니다.
        """
        y_pred = self.predict(x)
        return self.get_accuracy(y, y_pred)


    def get_params(self):
        """
        본 함수는 수정할 필요 없는 함수입니다.
        5fold-cross validation을 수행하기 위하여 필요한 함수입니다.
        """
        return {
            "n_iters":self.n_iters,
            "lr" :self.lr,
            "lambda_param" :self.lambda_param,
            "random_seed" :self.random_seed
            }


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])